# SYB scoring algorithm Demo-Notebook (v1 scoring algorithm prototype)

We collect here all the descriptions of the various algorithms we consider.

### Notation:

- $G = (V(G), E(G))$ is the considered graph of users, where the vertices $V(G)$  represent the users and the edges $E(G)$ the fact that two vertices vouched for each other.
- $G$ is represented as an undirected graph to show the process in a simplified setting.
- We think of $V(G)$ as the list $\{0,1,2,\ldots, |V(G)|-1\}$, where we denote by $|V(G)|$ the cardinality of this set.
- We call a vector $x\in \mathbb{R}_{\geq 0}^{|V(G)|}$, that is, a vector with $|V(G)|$ non-negative real entries, a *score vector*. It represents the values of the corresponding users.

# Degree inversion algorithm

The first algorithm we implement is based on the formula 

$$
y_i:=\sum\limits_{(j,i)\in E(G)}\frac{x_j}{\deg(j)}
$$

where $x$ is the old score vector, $y$ is the new score vector, and the graph $G$ has $j$ linked to $i$ if the two nodes vouched for each other (in the general application setting, they will be linked if they both have the required balance and have both vouched for each other).

## Code:

Google Drive folder: https://drive.google.com/drive/folders/1ELgmF11LuUB-9JZYFIHze92BLZDrd20_?usp=sharing

Code: https://drive.google.com/file/d/1hn_u5DrOejn6sP67odUjzrite7YPdhmD/view?usp=drive_link

### Code behaviour:

- on input two values, $n$ and $m$, generates a random graph ($G$) over $n$ vertices and $m$ edges (we consider undirected graphs and we allow at most one edge between any two given vertices).
- on input a vector on $n$ non-negative values (meant as the present score value of the vertices), say $x=[x_1,...,x_n]$, compute the next score values of the nodes with the above formula.
- if no vector $x$ is provided, initialize it as the vector $[1/n,1/n,\ldots,1/n]$.
- plot the graphs with initial values and also with the updated values.
- ask the user to a couple of vertices, say $i$ and $j$:
    - if the edge $(i,j)$ is already present, then ask for another couple, and repeat;
    - otherwise:
        - add the edge $(i,j)$ to the graph $G$,
        - compute the updated score, and
        - plot the new graph with the new values.

We can compute an example as follows, by specifying the number of vertices and edges. Note that, given $n$ vertices, we have at most $n\cdot(n-1)/2$ edges.

In [ ]:
import degree_score_algorithm_plot as algo
N_VERTICES = 10
M_EDGES = 15
G, updated_scores = algo.generate_graph_and_display(N_VERTICES, M_EDGES)

We can update the graph, adding an edge.

In [ ]:
algo.handle_user_edge_addition(G, updated_scores)